<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Converter-file" data-toc-modified-id="Converter-file-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Converter file</a></span><ul class="toc-item"><li><span><a href="#Oliver-Version-Working!" data-toc-modified-id="Oliver-Version-Working!-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Oliver Version Working!</a></span></li></ul></li><li><span><a href="#New-Feature-Development" data-toc-modified-id="New-Feature-Development-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>New Feature Development</a></span></li><li><span><a href="#Morphology" data-toc-modified-id="Morphology-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Morphology</a></span><ul class="toc-item"><li><span><a href="#MorphGNT:sblgnt-Data-Description" data-toc-modified-id="MorphGNT:sblgnt-Data-Description-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>MorphGNT:sblgnt Data Description</a></span><ul class="toc-item"><li><span><a href="#Adding-Part-of-Speech-sp" data-toc-modified-id="Adding-Part-of-Speech-sp-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Adding Part of Speech <code>sp</code></a></span></li></ul></li></ul></li><li><span><a href="#Additional-Features" data-toc-modified-id="Additional-Features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Additional Features</a></span></li><li><span><a href="#Converting-the-update-source-file" data-toc-modified-id="Converting-the-update-source-file-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Converting the update source file</a></span></li><li><span><a href="#Testing-New-Features" data-toc-modified-id="Testing-New-Features-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Testing New Features</a></span></li></ul></div>

# Converter file

## Oliver Version Working!

In [66]:
import os
import re
import collections
import json
import csv
# from glob import glob
from tf.fabric import Fabric
from tf.convert.walker import CV
# from tf.compose import modify

source_dirs = 'input' # "input" is the name of the input folder that contains the source file
output_dirs = 'output' # "output" is the name of the output folder to which the finished TF files will be dumped into

bo2book = {line.split()[0]:line.split()[1] for line in '''
NTt3 New_Testament
'''.split('\n') if line} # "OT" is the name of the file in the input folder AND "split()" splits at space

# patts = {'section': re.compile('(\d*):(\d*)\.(\d*)')}

def director(cv):
        
    '''
    Walks through NA1904 and triggers
    slot and node creation events.
    '''
        
    # process books in order
    for bo, book in bo2book.items():
        
        book_loc = os.path.join(source_dirs, f'{bo}.txt')
        
        print(f'\thandling {book_loc}...')
        
        with open(book_loc, 'r', encoding="utf8") as infile:
            text = [w for w in infile.read().split('\n') if w]
            
        this_book = cv.node('book')
            
        # keep track of when to trigger paragraph, chapter, and verse objects
        # para_track = 1 # keep counts of paragraphs
        prev_book = "Matt" # start at Genesis
        prev_chap = 1 # start at 1
        prev_verse = 1 # start at 1
        
        sentence_track = 1
        sentence_done = False
        clause_track = 1
        clause_done = False
        
        wrdnum = 0 # start at 0
        this_chap = cv.node('chapter')
        # this_para = cv.node('paragraph')
        this_verse = cv.node('verse')
        this_sentence = cv.node('sentence')
        this_clause = cv.node('clause')
        
        
        
        
        # iterate through words and construct objects
        for word in text:

            wrdnum += 1

            data = word.split('\t')
            # word_data, lemmas = data[:7], data[7:]

            word_data = data[:28] #the number here is the amount of columns
            morphology = ' '.join(data[28:]) #the number here is the amount of columns
            
            # segment out word data
            # bo_code, ref, brake, ketiv, qere, morph, strongs = word_data
            orig_order, book, chapter, verse, word, normalized, lemma, morph_form, morph_functional, strongs, lexeme_dict, gloss, sp, gn, nu, ps, case, tense, voice, mood, degree, extra, freq_lemma, bol_dict_abc, lemma_translit, abc_order, BOL_lemma_dict, BOL_gloss = word_data



            #try:
            #    verse = int(verse)
            #except ValueError:
            #    subverse = verse[-1:]
            #    verse = verse[:-1]

            if verse == "":
                print(f'{orig_order}: {verse} {subverse}')


            # strongs_lemma, anlex_lemma = ' '.join(lemmas).split('!') # reconstitute lemmas and split on !

            # chapt, verse, wrdnum = [int(v) for v in patts['section'].match(ref).groups()]

            # -- handle TF events --

            # detect book boundary
            if prev_book != book:

                # end subverse
                # cv.feature(this_subverse, subverse=prev_subverse)
                # cv.terminate(this_subverse)

                # end verse
                cv.feature(this_verse, verse=prev_verse)
                cv.terminate(this_verse)
                
                # end chapter
                cv.feature(this_chap, chapter=prev_chap)
                cv.terminate(this_chap)

                # end book
                cv.feature(this_book, book=prev_book)
                cv.terminate(this_book)
                
                # new book, chapter, verse, and subverse begin
                this_book = cv.node('book')
                prev_book = book
                this_chap = cv.node('chapter')
                prev_chap = chapter
                this_verse = cv.node('verse')
                prev_verse = verse
                #this_subverse = cv.node('subverse')
                #prev_subverse = subverse
                wrdnum = 1
            
            # detect chapter boundary
            elif prev_chap != chapter:

                # end subverse
                #cv.feature(this_subverse, subverse=prev_subverse)
                #cv.terminate(this_subverse)
                
                # end verse
                cv.feature(this_verse, verse=prev_verse)
                cv.terminate(this_verse)
                
                # end chapter
                cv.feature(this_chap, chapter=prev_chap)
                cv.terminate(this_chap)
                
                # new chapter, verse, and subverse begin
                this_chap = cv.node('chapter')
                prev_chap = chapter
                this_verse = cv.node('verse')
                prev_verse = verse
                #this_subverse = cv.node('subverse')
                #prev_subverse = subverse
                wrdnum = 1
            
            # detect verse boundary
            elif prev_verse != verse:

                # end subverse
                #cv.feature(this_subverse, subverse=prev_subverse)
                #cv.terminate(this_subverse)

                # end verse
                cv.feature(this_verse, verse=prev_verse)
                cv.terminate(this_verse)

                # new verse and subverse begin
                this_verse = cv.node('verse')
                prev_verse = verse
                #this_subverse = cv.node('subverse')
                #prev_subverse = subverse
                wrdnum = 1

            # detect subverse boundary
            #elif prev_subverse != subverse:
            #    cv.feature(this_subverse, subverse=prev_subverse)
            #    cv.terminate(this_subverse)
            #    this_subverse = cv.node('subverse')
            #    prev_subverse = subverse

                
            # detect paragraph boundary
            # if brake == 'P':
            #     cv.feature(this_para, para=para_track)
            #     cv.terminate(this_para)
            #     this_para = cv.node('paragraph') # start a new paragraph
            #     para_track += 1 # count paragraphs in the book
            
            
            if sentence_done:
                cv.feature(this_clause, clause=clause_track)
                cv.terminate(this_clause)
                cv.feature(this_sentence, sentence=sentence_track)
                cv.terminate(this_sentence)
                this_sentence = cv.node('sentence')
                sentence_track += 1
                this_clause = cv.node('clause')
                clause_track += 1

                sentence_done = False
                clause_done = False

            elif clause_done:
                cv.feature(this_clause, clause=clause_track)
                cv.terminate(this_clause)
                this_clause = cv.node('clause')
                clause_track += 1

                clause_done = False

            if text[-1:] == "." or text[-1:] == ";":
                sentence_done = True

            elif text[-1:] == "," or text[-1:] == "·":
                clause_done = True
            
            
                
            # make word object
            this_word = cv.slot()
            cv.feature(this_word, 
                        orig_order=orig_order,
                        book=book,
                        chapter=chapter,
                        verse=verse,
                        word=word,
                        normalized=normalized,
                        lemma=lemma,
                        morph_form=morph_form,
                        morph_functional=morph_functional,
                        strongs=strongs,
                        lexeme_dict=lexeme_dict,
                        gloss=gloss,
                        sp=sp,
                        gn=gn,
                        nu=nu,
                        ps=ps,
                        case=case,
                        tense=tense,
                        voice=voice,
                        mood=mood,
                        degree=degree,
                        extra=extra,
                        freq_lemma=freq_lemma,
                        bol_dict_abc=bol_dict_abc,
                        lemma_translit=lemma_translit,
                        abc_order=abc_order,
                        BOL_lemma_dict=BOL_lemma_dict,
                        BOL_gloss=BOL_gloss,
                       # ketiv=ketiv, 
                       # qere=qere, 
                       # strongs=strongs, 
                    #    str_lem=strongs_lemma.strip(),
                    #    anlex_lem=anlex_lemma.strip()
                      )
            cv.terminate(this_word)
        
        # end book and its objects
        # - end subverse
        #cv.feature(this_subverse, subverse=prev_subverse)
        #cv.terminate(this_subverse)
        
       # - end clause
        cv.feature(this_clause, clause=clause_track)
        cv.terminate(this_clause)

        # - end sentence
        cv.feature(this_sentence, sentence=sentence_track)
        cv.terminate(this_sentence)
        

        # - end verse
        cv.feature(this_verse, verse=prev_verse)
        cv.terminate(this_verse)
        
        # - end paragraph
        # cv.feature(this_para, para=para_track)
        # cv.terminate(this_para)
        
        # - end chapter
        cv.feature(this_chap, chapter=prev_chap)
        cv.terminate(this_chap)
        
        # - end book
        cv.feature(this_book, book=prev_book)
        cv.terminate(this_book)


slotType = 'word'
otext = {'fmt:text-orig-full':'{word} ',
         'sectionTypes':'book,chapter,verse',
         'sectionFeatures':'book,chapter,verse'}

generic = {'Name': 'NA1904',
           'Version': '1904',
           'Author': 'Nestle & Aland',
           'Editors': 'Ulrik Sandborg Petersen, Eliran Wong, Oliver Glanz, BOL',
           'Converter': 'Oliver Glanz', 
           'Source:':'https://github.com/biblicalhumanities/Nestle1904/blob/master/morph/Nestle1904.csv',
           'Note':'?'}

intFeatures = {'chapter', 'verse'}

featureMeta = {
                'orig_order': {'description': 'word order within corpus'},
                'book': {'description': 'book'},
                'chapter': {'description': 'book'},
                'verse': {'description': 'verse'},
                'sentence': {'description': 'sentence'},
                'clause': {'description': 'clause'},
                'word': {'description': 'word as it appears in the text'},
                'normalized': {'description': 'normalized word'},
                'lemma': {'description': 'lemma'},
                'morph_form': {'description': 'formal morphology coding'},
                'morph_functional': {'description': 'functional morphology coding'},
                'strongs': {'description': 'Strong\'s numbers'},
                'lexeme_dict': {'description': 'dictionary form of lemma'},
                'gloss': {'description': 'English translation'},
                'sp': {'description': 'part of speech'},
                'gn': {'description': 'gender'},
                'nu': {'description': 'number'},
                'ps': {'description': 'person'},
                'case': {'description': 'case'},
                'tense': {'description': 'tense'},
                'voice': {'description': 'voice'},
                'mood': {'description': 'mood'},
                'degree': {'description': 'degree'},
                'extra': {'description': 'extra notes'},
                'freq_lemma': {'description': 'word frequency in corpus'},
                'bol_dict_abc': {'description': 'BOL based alphabetic order of words'},
                'lemma_translit': {'description': 'lemma transliteration'},
                'abc_order': {'description': 'NA1904 alphabetic order of words'},
                'BOL_lemma_dict': {'description': 'BOL based dictionary form'},
                'BOL_gloss': {'description': 'BOL based English gloss'},
               #'subverse': {'description': 'Subverse information'},
               # 'para': {'description': 'A paragraph number'},
               # 'ketiv': {'descrption': 'The text as it is written in the printed Tischendorf'},
               # 'qere': {'description': 'The text as the editor thinks it should have been'},
               # 'strongs': {'description': 'A word\'s number in Strongs'},
               # 'str_lem': {'description': 'Word lemma that corresponds to The NEW Strong\'sComplete Dictionary of Bible Words'},
               # 'anlex_lem': {'description': 'Word lemma that corresponds to Friberg, Friberg and Miller\'s ANLEX'}
              }


# configure metadata/output
version = '1904'
generic['Version'] = version

output = os.path.join(output_dirs, version)

print(f'Processing Version {version}')
output_dir = output_dirs.format(version=version)

TF = Fabric(locations=output_dir, silent=True)
cv = CV(TF)

cv.walk(director,
                slotType,
                otext=otext,
                generic=generic,
                intFeatures=intFeatures,
                featureMeta=featureMeta,
                warn=True,
                force=False,)

Processing Version 1904
  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |     0.00s No structure nodes will be set up
   |   SECTION   TYPES:    book, chapter, verse
   |   SECTION   FEATURES: book, chapter, verse
   |   STRUCTURE TYPES:    
   |   STRUCTURE FEATURES: 
   |   TEXT      FEATURES:
   |      |   text-orig-full       word
   |     0.00s OK
   |     0.00s Following director... 
	handling input\NTt3.txt...
   |     2.66s "edge" actions: 0
   |     2.66s "feature" actions: 146013
   |     2.67s "node" actions: 8234
   |     2.67s "resume" actions: 0
   |     2.67s "slot" actions: 137779
   |     2.67s "terminate" actions: 146013
   |         27 x "book" node 
   |        261 x "chapter" node 
   |          1 x "clause" node 
   |          1 x "sentence" node 
   |       7944 x "verse" node 
   |     137779 x "word" node  = slot type
   |     146013 nodes of all types
   |     2.69s OK
   |     0.00s Removing unlinked nodes .

True

In [54]:
%load_ext autoreload
%autoreload 2

In [55]:
# First, I have to laod different modules that I use for analyzing the data and for plotting:
import sys, os, collections
import pandas as pd
import numpy as np
import re
import csv
import seaborn as sns
import matplotlib.pyplot as plt; plt.rcdefaults()
from matplotlib.pyplot import figure
from collections import Counter

# Second, I have to load the Text Fabric app
from tf.fabric import Fabric
from tf.app import use

In [68]:
#LXX = use('CCATLXX/tf/1994_v1', hoist=globals())
NA = use('NA1904/tf/1904_v3', hoist=globals())

This is Text-Fabric 9.1.11
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

33 features found and 0 ignored
   |     0.09s T otype                from D:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/NA1904/tf/1904_v3
   |     0.70s T oslots               from D:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/NA1904/tf/1904_v3
   |     1.07s T verse                from D:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/NA1904/tf/1904_v3
   |     0.99s T book                 from D:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/NA1904/tf/1904_v3
   |     1.16s T word                 from D:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/NA1904/tf/1904_v3
   |     0.88s T chapter              from D:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/NA1904/tf/1904_v3
   |      |     0.01s C __levels__           from otype, oslots, otext
   |      |     0.81s C __order__            from otype, oslots, __levels__
   |      |    

In [72]:
Search0 = '''
book book=Matt
  chapter chapter=1
    verse verse=1
      word
'''
Search0 = NA.search(Search0)
NA.show(Search0, start=1, end=1, condensed=True, colorMap={1:'pink'}, extraFeatures={'clause','sentence','lemma','morph_form','morph_functional','normalized','strongs','lexeme_dict','gloss', 'sp', 'gn', 'nu', 'ps', 'case', 'tense', 'voice', 'mood', 'degree', 'extra', 'freq_lemma', 'bol_dict_abc', 'lemma_translit', 'abc_order', 'BOL_lemma_dict','BOL_gloss'})

  0.16s 0 results


In [60]:
Search1 = '''
verse book=Matt chapter=1 verse=1
      word
'''
Search1 = NA.search(Search1)
NA.show(Search1, start=1, end=1, condensed=True, colorMap={1:'pink'}, extraFeatures={'lemma'})

  0.04s 0 results


In [8]:
Search2 = '''
book book=Gen
  chapter chapter=1
    verse verse=1
        word word* lex_utf8 g_cons_utf8 morphology* translit_SBL
'''
Search2 = LXX.search(Search2)
LXX.show(Search2, start=1, end=1, condensed=True, colorMap={1:'pink'}, extraFeatures={'subverse'})

  2.74s 10 results


# New Feature Development

# Morphology
## MorphGNT:sblgnt Data Description
Here: https://github.com/morphgnt/sblgnt


In [41]:
featureprep=pd.read_excel('d:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/Nestle1904_source_v1.0.xlsx')
pd.set_option('display.max_columns', 50)
featureprep.head(5)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN


In [42]:
morph=featureprep

### Adding Part of Speech `sp`

In [43]:
def spconditions(row):
    if re.search('^A-.*', str(row)):
        return 'adj'    
    if re.search('^C.*', str(row)):
        return 'conj'    
    if re.search('^D.*', str(row)):
        return 'adv'     
    if re.search('^I.*', str(row)):
        return 'interj'    
    if re.search('^N-.*', str(row)):
        return 'noun'      
    if re.search('^P.*', str(row)):
        return 'prep'
    if re.search('^T-.*', str(row)):
        return 'art'
    if re.search('^C-.*', str(row)):
        return 'pron-recip' 
    if re.search('^RI.*', str(row)):
        return 'pron-inter'
    if re.search('^P-.*', str(row)):
        return 'pron-prs'
    if re.search('^D-.*', str(row)):
        return 'pron-demon'       
    if re.search('^K-.*', str(row)):
        return 'pron-correl'
    if re.search('^I-.*', str(row)):
        return 'pron-interrog'   
    if re.search('^X-.*', str(row)):
        return 'pron-indefi'        
    if re.search('^R-.*', str(row)):
        return 'pron-rela' 
    if re.search('^Q-.*', str(row)):
        return 'pron-correl-interrog'        
    if re.search('^F-.*', str(row)):
        return 'pron-reflex'        
    if re.search('^S-.*', str(row)):
        return 'pron-posses'        
    if re.search('^ADV.*', str(row)):
        return 'adv'        
    if re.search('^CONJ.*', str(row)):
        return 'conj'     
    if re.search('^COND.*', str(row)):
        return 'cond'     
    if re.search('^PRT.*', str(row)):
        return 'particle'     
    if re.search('^PREP.*', str(row)):
        return 'preposition'     
    if re.search('^INJ.*', str(row)):
        return 'interjection'     
    if re.search('^ARAM.*', str(row)):
        return 'aramaic'     
    if re.search('^HEB.*', str(row)):
        return 'hebrew'     
    if re.search('^N-PRI.*', str(row)):
        return 'proper noun'     
    if re.search('^A-NUI.*', str(row)):
        return 'numeral'     
    if re.search('^N-LI.*', str(row)):
        return 'letter'     
    if re.search('^N-OI.*', str(row)):
        return 'noun_other'     
    if re.search('^PUNCT.*', str(row)):
        return 'punctuation'     
    if re.search('^V-.*', str(row)):
        return 'verb'    

    else:
        return ''

In [44]:
morph['sp']=morph['morph_form'].apply(spconditions)
morph.head(20)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb


In [45]:
def gender(row):
        if re.search('.*F$', str(row)):
            return 'f'
        if re.search('.*M$', str(row)):
            return 'm'
        if re.search('.*N$', str(row)):
            return 'n'
        else:
            return ''

In [46]:
morph['gn']=morph['morph_form'].apply(gender)
morph.head(20)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,


In [47]:
def number(row):
        if re.search('.*S.$', str(row)):
            return 'sg'  
        if re.search('.*P.$', str(row)):
            return 'pl'   
        else:
            return ''

In [48]:
morph['nu']=morph['morph_form'].apply(number)
morph.head(50)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,


In [49]:
def person(row):
        if re.search('.*1.$', str(row)):
            return 'p1'   
        if re.search('.*2.$', str(row)):
            return 'p2'   
        if re.search('.*3.$', str(row)):
            return 'p3'      
        else:
            return ''

In [50]:
morph['ps']=morph['morph_form'].apply(person)
morph.head(50)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu,ps
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg,
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg,
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg,
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg,
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,,p3


In [51]:
def case(row):
        if re.search('[NATPRCDKIXQFS]-N.*$', str(row)):
            return 'nominative'   
        if re.search('[NATPRCDKIXQFS]-G.*$', str(row)):
            return 'genitive'
        if re.search('[NATPRCDKIXQFS]-D.*$', str(row)):
            return 'dative'
        if re.search('[NATPRCDKIXQFS]-A.*$', str(row)):
            return 'accusative'
        if re.search('[NATPRCDKIXQFS]-V.*$', str(row)):
            return 'vocative'        
        else:
            return ''

In [52]:
morph['case']=morph['morph_form'].apply(case)
morph.head(50)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu,ps,case
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg,,nominative
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg,,genitive
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg,,genitive
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg,,genitive
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,,,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,,p3,


In [53]:
def tense(row):
        if re.search('V-P.*', str(row)):
            return 'present'    
        if re.search('V-I.*', str(row)):
            return 'imperfect'   
        if re.search('V-F.*', str(row)):
            return 'future'   
        if re.search('V-2F.*', str(row)):
            return '2ndfuture'   
        if re.search('V-A.*', str(row)):
            return 'aorist'   
        if re.search('V-2A.*', str(row)):
            return '2ndaorist'   
        if re.search('V-R.*', str(row)):
            return 'perfect'   
        if re.search('V-2R.*', str(row)):
            return '2ndperfect'   
        if re.search('V-L.*', str(row)):
            return 'pluperfect'   
        if re.search('V-2L.*', str(row)):
            return '2ndpluperfect'   
        if re.search('V-X.*', str(row)):
            return 'no_tense_stated'         
        else:
            return ''

In [54]:
morph['tense']=morph['morph_form'].apply(tense)
morph.head(50)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu,ps,case,tense
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg,,nominative,
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg,,genitive,
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg,,genitive,
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg,,genitive,
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,,,,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,,p3,,aorist


In [55]:
def voice(row):
        if re.search('V-.A.*$', str(row)):
            return 'active'
        if re.search('V-.M.*$', str(row)):
            return 'middle'     
        if re.search('V-.P.*$', str(row)):
            return 'passive'
        if re.search('V-.E.*$', str(row)):
            return 'middle_or_passive'        
        if re.search('V-.D.*$', str(row)):
            return 'middle_deponent'
        if re.search('V-.O.*$', str(row)):
            return 'passive_deponent'
        if re.search('V-.N.*$', str(row)):
            return 'middle_or_passive_deponent'        
        if re.search('V-.Q.*$', str(row)):
            return 'impersonal_active'        
        if re.search('V-.X.*$', str(row)):
            return 'no_voice'              
        else:
            return ''

In [56]:
morph['voice']=morph['morph_form'].apply(voice)
morph.head(50)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu,ps,case,tense,voice
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg,,nominative,,
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg,,genitive,,
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg,,genitive,,
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg,,genitive,,
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,,,,,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,,p3,,aorist,active


In [57]:
def mood(row):
        if re.search('V-..I.*$', str(row)):
            return 'indicative'
        if re.search('V-..S.*$', str(row)):
            return 'subjunctive'     
        if re.search('V-..O.*$', str(row)):
            return 'optative'
        if re.search('V-..M.*$', str(row)):
            return 'imperative'        
        if re.search('V-..N.*$', str(row)):
            return 'infinitive'
        if re.search('V-..P.*$', str(row)):
            return 'participle'
        if re.search('V-..R.*$', str(row)):
            return 'imperative_participle'                  
        else:
            return ''

In [58]:
morph['mood']=morph['morph_form'].apply(mood)
morph.head(50)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu,ps,case,tense,voice,mood
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg,,nominative,,,
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg,,genitive,,,
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg,,genitive,,,
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg,,genitive,,,
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,,
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,,,,,,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,,
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,,p3,,aorist,active,indicative


In [59]:
def degree(row):
        if re.search('.*-C$', str(row)):
            return 'comparative'
        if re.search('.*-S$', str(row)):
            return 'superlative'
        else:
            return ''

In [60]:
morph['degree']=morph['morph_form'].apply(degree)
morph.head(50)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu,ps,case,tense,voice,mood,degree
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg,,nominative,,,,
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg,,genitive,,,,
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg,,genitive,,,,
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg,,genitive,,,,
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,,,
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,,,,,,,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,,,
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,,,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,,,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,,p3,,aorist,active,indicative,


In [61]:
def extra(row):
        if re.search('.*-ABB$', str(row)):
            return 'abbreviated'
        if re.search('.*-I$', str(row)):
            return 'interrogative'
        if re.search('.*-N$', str(row)):
            return 'negative'
        if re.search('.*-ATT$', str(row)):
            return 'attic'    
        if re.search('.*-P$', str(row)):
            return 'particle_attached'
        if re.search('.*-K$', str(row)):
            return 'crasis'        
        else:
            return ''

In [62]:
morph['extra']=morph['morph_form'].apply(extra)
morph.head(50)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu,ps,case,tense,voice,mood,degree,extra
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg,,nominative,,,,,
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg,,genitive,,,,,
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg,,genitive,,,,,
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg,,genitive,,,,,
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,,,,
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,,,,,,,,
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,,,,
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,,,,
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,,,,
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,,p3,,aorist,active,indicative,,


# Additional Features

In [6]:
# First, I have to laod different modules that I use for analyzing the data and for plotting:
import sys, os, collections
import pandas as pd
import numpy as np
import re
import csv
import seaborn as sns
import matplotlib.pyplot as plt; plt.rcdefaults()
from matplotlib.pyplot import figure
from collections import Counter

# Second, I have to load the Text Fabric app
from tf.fabric import Fabric
from tf.app import use

In [7]:
featureadd=pd.read_excel('d:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/Nestle1904_source_v1.1.xlsx')
pd.set_option('display.max_columns', 50)
featureadd.head(5)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,lexeme_dict,gloss,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma,bol_dict_abc
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,nominative,NaN,NaN,NaN,NaN,NaN,10,969
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",noun,f,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,5,1070
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,913,2399
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,529,5358
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,376,5026


In [8]:
from unidecode import unidecode

In [9]:
featureadd['lemma_translit']=featureadd['lemma'].apply(unidecode)
featureadd.head(5)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,lexeme_dict,gloss,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma,bol_dict_abc,lemma_translit
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,nominative,NaN,NaN,NaN,NaN,NaN,10,969,biblos
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",noun,f,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,5,1070,genesis
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,913,2399,Iesous
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,529,5358,Khristos
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,376,5026,uios


In [10]:
ABC1=featureadd[['orig_order','lemma']]
ABC1.head(5)

,orig_order,lemma
0,1,βίβλος
1,2,γένεσις
2,3,Ἰησοῦς
3,4,Χριστός
4,5,υἱός


In [11]:
ABC1.describe()

,orig_order
count,137779.000000
mean,68890.000000
std,39773.515707
min,1.000000
25%,34445.500000
50%,68890.000000
75%,103334.500000
max,137779.000000


In [12]:
ABCdict = ABC1.drop_duplicates(['lemma']).sort_values(by='lemma', ascending=[True])
ABCdict.head(10)

,orig_order,lemma
68648,68649,Αἰγύπτιος
69803,69804,Αἰθίοψ
70630,70631,Αἰνέας
50710,50711,Αἰνών
679,680,Αἴγυπτος
30771,30772,Αὐγοῦστος
171,172,Βαβυλών
124418,124419,Βαλαάμ
128732,128733,Βαλάκ
17192,17193,Βαραββᾶς


In [14]:
ABCdict.to_excel('d:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/feature-dev/ABC1order.xlsx')

In [15]:
ABC2=pd.read_excel('d:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/feature-dev/ABC2order.xlsx')
pd.set_option('display.max_columns', 50)
ABC2.head(10)

,orig_order,lemma,abc_order
0,1,βίβλος,965
1,2,γένεσις,1067
2,3,Ἰησοῦς,2387
3,4,Χριστός,5326
4,5,υἱός,4995
5,6,Δαυίδ,1148
6,8,Ἀβραάμ,10
7,10,γεννάω,1069
8,11,ὁ,3418
9,12,Ἰσαάκ,2424


In [16]:
ABC2=ABC2.drop(['orig_order'], axis=1)
ABC2.head()

,lemma,abc_order
0,βίβλος,965
1,γένεσις,1067
2,Ἰησοῦς,2387
3,Χριστός,5326
4,υἱός,4995


In [17]:
featureadd.describe()

,orig_order,chapter,verse,freq_lemma,bol_dict_abc
count,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000
mean,68890.000000,10.219206,18.719936,4187.035100,2666.125157
std,39773.515707,7.211703,13.606559,6795.931191,1332.673002
min,1.000000,1.000000,1.000000,1.000000,1.000000
25%,34445.500000,4.000000,8.000000,84.000000,1498.000000
50%,68890.000000,9.000000,16.000000,668.000000,2737.000000
75%,103334.500000,15.000000,26.000000,2892.000000,3581.500000
max,137779.000000,28.000000,99.000000,19783.000000,5433.000000


In [18]:
featureadd=pd.merge (featureadd, ABC2,
                      on='lemma',
                      how='outer')
featureadd.head(5)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,lexeme_dict,gloss,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma,bol_dict_abc,lemma_translit,abc_order
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,nominative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
1,26411,Mark,12,26,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
2,31677,Luke,3,4,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
3,45622,Luke,20,42,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
4,65061,Acts,1,20,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965


In [19]:
featureadd.describe()

,orig_order,chapter,verse,freq_lemma,bol_dict_abc,abc_order
count,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000
mean,68890.000000,10.219206,18.719936,4187.035100,2666.125157,2651.773434
std,39773.515707,7.211703,13.606559,6795.931191,1332.673002,1323.818004
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,34445.500000,4.000000,8.000000,84.000000,1498.000000,1492.000000
50%,68890.000000,9.000000,16.000000,668.000000,2737.000000,2723.000000
75%,103334.500000,15.000000,26.000000,2892.000000,3581.500000,3561.500000
max,137779.000000,28.000000,99.000000,19783.000000,5433.000000,5401.000000


In [20]:
featureaddstage2 = featureadd
featureaddstage2.head(5)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,lexeme_dict,gloss,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma,bol_dict_abc,lemma_translit,abc_order
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,nominative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
1,26411,Mark,12,26,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
2,31677,Luke,3,4,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
3,45622,Luke,20,42,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
4,65061,Acts,1,20,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965


In [21]:
featureaddstage2.describe()

,orig_order,chapter,verse,freq_lemma,bol_dict_abc,abc_order
count,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000
mean,68890.000000,10.219206,18.719936,4187.035100,2666.125157,2651.773434
std,39773.515707,7.211703,13.606559,6795.931191,1332.673002,1323.818004
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,34445.500000,4.000000,8.000000,84.000000,1498.000000,1492.000000
50%,68890.000000,9.000000,16.000000,668.000000,2737.000000,2723.000000
75%,103334.500000,15.000000,26.000000,2892.000000,3581.500000,3561.500000
max,137779.000000,28.000000,99.000000,19783.000000,5433.000000,5401.000000


In [22]:
featureaddstage2["freq_lemma"]=featureaddstage2.groupby(["lemma"])["lemma"].transform("count")
featureaddstage2.head(5)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,lexeme_dict,gloss,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma,bol_dict_abc,lemma_translit,abc_order
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,nominative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
1,26411,Mark,12,26,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
2,31677,Luke,3,4,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
3,45622,Luke,20,42,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
4,65061,Acts,1,20,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965


In [23]:
featureaddstage2.describe()

,orig_order,chapter,verse,freq_lemma,bol_dict_abc,abc_order
count,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000
mean,68890.000000,10.219206,18.719936,4187.035100,2666.125157,2651.773434
std,39773.515707,7.211703,13.606559,6795.931191,1332.673002,1323.818004
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,34445.500000,4.000000,8.000000,84.000000,1498.000000,1492.000000
50%,68890.000000,9.000000,16.000000,668.000000,2737.000000,2723.000000
75%,103334.500000,15.000000,26.000000,2892.000000,3581.500000,3561.500000
max,137779.000000,28.000000,99.000000,19783.000000,5433.000000,5401.000000


In [24]:
featureaddstage2.sort_values(['orig_order'], ascending=True).head(10)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,lexeme_dict,gloss,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma,bol_dict_abc,lemma_translit,abc_order
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,nominative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965
10,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",noun,f,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,5,1070,genesis,1067
15,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,913,2399,Iesous,2387
928,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,529,5358,Khristos,5326
1457,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,376,5026,uios,4995
1833,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,noun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,1151,Dauid,1148
1458,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,376,5026,uios,4995
1892,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,noun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,10,Abraam,10
1893,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,noun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,10,Abraam,10
1965,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",verb,NaN,NaN,p3,NaN,aorist,active,indicative,NaN,NaN,97,1072,gennao,1069


In [25]:
featureaddstage2.describe()

,orig_order,chapter,verse,freq_lemma,bol_dict_abc,abc_order
count,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000
mean,68890.000000,10.219206,18.719936,4187.035100,2666.125157,2651.773434
std,39773.515707,7.211703,13.606559,6795.931191,1332.673002,1323.818004
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,34445.500000,4.000000,8.000000,84.000000,1498.000000,1492.000000
50%,68890.000000,9.000000,16.000000,668.000000,2737.000000,2723.000000
75%,103334.500000,15.000000,26.000000,2892.000000,3581.500000,3561.500000
max,137779.000000,28.000000,99.000000,19783.000000,5433.000000,5401.000000


In [26]:
BOLgreekDICT=pd.read_excel('d:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_SBLGNT/feature-dev/NA1904_dictionary_v1.0.xlsx')
pd.set_option('display.max_columns', 50)
BOLgreekDICT.head(10)

,orig abc order,Occurrences,Lexeme,orig abc order.1,Lexeme_dict,Strong's number,Strong's unreliable?,gloss
0,1,5,Ἀαρών,1,"Ἀαρών, ὁ",2,no,Aaron
1,2,1,Ἀβαδδών,2,"Ἀβαδδών, ὁ",3,no,Abaddon
2,3,1,ἀβαρής,3,"ἀβαρής, -ές",4,no,not burdensome
3,4,3,ἀββά,4,"ἀββά, ὁ",5,no,Father
4,5,4,Ἅβελ,5,"Ἅβελ, ὁ",6,no,Abel
5,6,3,Ἀβιά,6,"Ἀβιά, ὁ",7,no,Abijah
6,7,1,Ἀβιαθάρ,7,"Ἀβιαθάρ, ὁ",8,no,Abiathar
7,8,1,Ἀβιληνή,8,"Ἀβιληνή, -ῆς, ἡ",9,no,Abilene
8,9,2,Ἀβιούδ,9,"Ἀβιούδ, ὁ",10,no,Abiud
9,10,73,Ἀβραάμ,10,"Ἀβραάμ, ὁ",11,no,Abraham


In [27]:
BOLgreekDICT=BOLgreekDICT[['Lexeme','Lexeme_dict', 'gloss']]
BOLgreekDICT.head(10)

,Lexeme,Lexeme_dict,gloss
0,Ἀαρών,"Ἀαρών, ὁ",Aaron
1,Ἀβαδδών,"Ἀβαδδών, ὁ",Abaddon
2,ἀβαρής,"ἀβαρής, -ές",not burdensome
3,ἀββά,"ἀββά, ὁ",Father
4,Ἅβελ,"Ἅβελ, ὁ",Abel
5,Ἀβιά,"Ἀβιά, ὁ",Abijah
6,Ἀβιαθάρ,"Ἀβιαθάρ, ὁ",Abiathar
7,Ἀβιληνή,"Ἀβιληνή, -ῆς, ἡ",Abilene
8,Ἀβιούδ,"Ἀβιούδ, ὁ",Abiud
9,Ἀβραάμ,"Ἀβραάμ, ὁ",Abraham


In [28]:
BOLgreekDICT = BOLgreekDICT.rename({'Lexeme':'lemma', 'Lexeme_dict':'BOL_lemma_dict', 'gloss':'BOL_gloss'}, axis=1)
BOLgreekDICT.head(5)

,lemma,BOL_lemma_dict,BOL_gloss
0,Ἀαρών,"Ἀαρών, ὁ",Aaron
1,Ἀβαδδών,"Ἀβαδδών, ὁ",Abaddon
2,ἀβαρής,"ἀβαρής, -ές",not burdensome
3,ἀββά,"ἀββά, ὁ",Father
4,Ἅβελ,"Ἅβελ, ὁ",Abel


In [35]:
BOLgreekDICT.describe()

,lemma,BOL_lemma_dict,BOL_gloss
count,5433,5335,5433
unique,5400,5297,5172
top,ταχύς,"ταχύς, -εῖα","put on, clothe"
freq,4,4,4


In [41]:
featureaddstage3=featureaddstage2

In [42]:
featureaddstage3.describe()

,orig_order,chapter,verse,freq_lemma,bol_dict_abc,abc_order
count,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000
mean,68890.000000,10.219206,18.719936,4187.035100,2666.125157,2651.773434
std,39773.515707,7.211703,13.606559,6795.931191,1332.673002,1323.818004
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,34445.500000,4.000000,8.000000,84.000000,1498.000000,1492.000000
50%,68890.000000,9.000000,16.000000,668.000000,2737.000000,2723.000000
75%,103334.500000,15.000000,26.000000,2892.000000,3581.500000,3561.500000
max,137779.000000,28.000000,99.000000,19783.000000,5433.000000,5401.000000


In [44]:
#featureaddstage3.merge(BOLgreekDICT, how='left', on=['lemma'])
#featureaddstage3.head(5)

featureaddstage4=pd.merge (featureaddstage3, BOLgreekDICT,
                      on='lemma',
                      how='left')
featureaddstage4.head(5)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,lexeme_dict,gloss,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma,bol_dict_abc,lemma_translit,abc_order,BOL_lemma_dict,BOL_gloss
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,nominative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965,"βίβλος, -ου, ἡ","written book, roll, or volume"
1,26411,Mark,12,26,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965,"βίβλος, -ου, ἡ","written book, roll, or volume"
2,31677,Luke,3,4,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965,"βίβλος, -ου, ἡ","written book, roll, or volume"
3,45622,Luke,20,42,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965,"βίβλος, -ου, ἡ","written book, roll, or volume"
4,65061,Acts,1,20,βίβλῳ,βίβλῳ,βίβλος,N-DSF,N-DSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,dative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965,"βίβλος, -ου, ἡ","written book, roll, or volume"


In [47]:
featureaddstage4.describe()

,orig_order,chapter,verse,freq_lemma,bol_dict_abc,abc_order
count,140309.000000,140309.000000,140309.000000,140309.000000,140309.000000,140309.000000
mean,68806.814246,10.228645,18.723467,4115.735377,2672.532069,2658.132693
std,39765.846252,7.213326,13.609302,6754.935029,1330.664991,1321.815113
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,34341.000000,4.000000,8.000000,84.000000,1498.000000,1492.000000
50%,68806.000000,9.000000,16.000000,647.000000,2759.000000,2745.000000
75%,103224.000000,15.000000,26.000000,2892.000000,3591.000000,3571.000000
max,137779.000000,28.000000,99.000000,19783.000000,5433.000000,5401.000000


In [49]:
featureaddstage4 = featureaddstage4.drop_duplicates(['orig_order']).sort_values(by='orig_order', ascending=[True])
featureaddstage4.head(10)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,lexeme_dict,gloss,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma,bol_dict_abc,lemma_translit,abc_order,BOL_lemma_dict,BOL_gloss
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",noun,f,sg,NaN,nominative,NaN,NaN,NaN,NaN,NaN,10,969,biblos,965,"βίβλος, -ου, ἡ","written book, roll, or volume"
10,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",noun,f,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,5,1070,genesis,1067,"γένεσις, -εως, ἡ","birth, lineage"
15,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,913,2399,Iesous,2387,Ἰησοῦς,Jesus
928,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,529,5358,Khristos,5326,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ"
1457,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,376,5026,uios,4995,"υἱός, -οῦ, ὁ","son, descendent"
1833,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,noun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,1151,Dauid,1148,"Δαυίδ, ὁ",David
1458,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",noun,m,sg,NaN,genitive,NaN,NaN,NaN,NaN,NaN,376,5026,uios,4995,"υἱός, -οῦ, ὁ","son, descendent"
1892,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,noun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,10,Abraam,10,"Ἀβραάμ, ὁ",Abraham
1893,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,noun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73,10,Abraam,10,"Ἀβραάμ, ὁ",Abraham
1965,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",verb,NaN,NaN,p3,NaN,aorist,active,indicative,NaN,NaN,97,1072,gennao,1069,γεννάω,"beget, bring forth, give birth to"


In [50]:
featureaddstage4.describe()

,orig_order,chapter,verse,freq_lemma,bol_dict_abc,abc_order
count,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000,137779.000000
mean,68890.000000,10.219206,18.719936,4187.035100,2666.125157,2651.773434
std,39773.515707,7.211703,13.606559,6795.931191,1332.673002,1323.818004
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,34445.500000,4.000000,8.000000,84.000000,1498.000000,1492.000000
50%,68890.000000,9.000000,16.000000,668.000000,2737.000000,2723.000000
75%,103334.500000,15.000000,26.000000,2892.000000,3581.500000,3561.500000
max,137779.000000,28.000000,99.000000,19783.000000,5433.000000,5401.000000


In [51]:
featureaddstage4.to_excel('d:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/Nestle1904_source_v1.3.xlsx')

In [428]:
morph['orig_order'] = frequencyadd.index +1
frequencyadd['lemma']=frequencyadd[0]
frequencyadd=frequencyadd[['orig_order','lemma']]
frequencyadd.head(5)

,orig_order,lemma
0,1,βίβλος
1,2,γένεσις
2,3,Ἰησοῦς
3,4,Χριστός
4,5,υἱός


In [64]:
morph["freq_lemma"]=morph.groupby(["lemma"])["lemma"].transform("count")
#("count") is actually utilizing the 'count' function!
morph.head(20)

,orig_order,book,chapter,verse,word,normalized,lemma,morph_form,morph_functional,strongs,Lexeme_dict,gloss,occurrence,Unnamed: 13,sp,gn,nu,ps,case,tense,voice,mood,degree,extra,freq_lemma
0,1,Matt,1,1,Βίβλος,Βίβλος,βίβλος,N-NSF,N-NSF,976,"βίβλος, -ου, ἡ","written book, roll, or volume",10,NaN,noun,f,sg,,nominative,,,,,,10
1,2,Matt,1,1,γενέσεως,γενέσεως,γένεσις,N-GSF,N-GSF,1078,"γένεσις, -εως, ἡ","birth, lineage",5,NaN,noun,f,sg,,genitive,,,,,,5
2,3,Matt,1,1,Ἰησοῦ,Ἰησοῦ,Ἰησοῦς,N-GSM,N-GSM,2424,Ἰησοῦς,Jesus,913,NaN,noun,m,sg,,genitive,,,,,,913
3,4,Matt,1,1,Χριστοῦ,Χριστοῦ,Χριστός,N-GSM,N-GSM,5547,"Χριστός, -οῦ, ὁ","anointed, Messiah, Christ",529,NaN,noun,m,sg,,genitive,,,,,,529
4,5,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,,,,,376
5,6,Matt,1,1,Δαυεὶδ,Δαυείδ,Δαυίδ,N-PRI,N-PRI,1138,"Δαυίδ, ὁ",David,59,NaN,noun,,,,,,,,,,59
6,7,Matt,1,1,υἱοῦ,υἱοῦ,υἱός,N-GSM,N-GSM,5207,"υἱός, -οῦ, ὁ","son, descendent",376,NaN,noun,m,sg,,genitive,,,,,,376
7,8,Matt,1,1,Ἀβραάμ.,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,,,,,73
8,9,Matt,1,2,Ἀβραὰμ,Ἀβραάμ,Ἀβραάμ,N-PRI,N-PRI,11,"Ἀβραάμ, ὁ",Abraham,73,NaN,noun,,,,,,,,,,73
9,10,Matt,1,2,ἐγέννησεν,ἐγέννησεν,γεννάω,V-AAI-3S,V-AAI-3S,1080&5656,γεννάω,"beget, bring forth, give birth to",97,NaN,verb,,,p3,,aorist,active,indicative,,,97


In [65]:
morph.to_excel('d:/OneDrive/1200_AUS-research/Fabric-TEXT/0_data_NA1904/Nestle1904_source_v1.1.xlsx')

# Converting the update source file

In [74]:
import os
import re
import collections
import json
import csv
# from glob import glob
from tf.fabric import Fabric
from tf.convert.walker import CV
# from tf.compose import modify

source_dirs = 'input' # "input" is the name of the input folder that contains the source file
output_dirs = 'output' # "output" is the name of the output folder to which the finished TF files will be dumped into

bo2book = {line.split()[0]:line.split()[1] for line in '''
NTt2 New_Testament
'''.split('\n') if line} # "OT" is the name of the file in the input folder AND "split()" splits at space

# patts = {'section': re.compile('(\d*):(\d*)\.(\d*)')}

def director(cv):
        
    '''
    Walks through NA1904 and triggers
    slot and node creation events.
    '''
        
    # process books in order
    for bo, book in bo2book.items():
        
        book_loc = os.path.join(source_dirs, f'{bo}.txt')
        
        print(f'\thandling {book_loc}...')
        
        with open(book_loc, 'r', encoding="utf8") as infile:
            text = [w for w in infile.read().split('\n') if w]
            
        this_book = cv.node('book')
            
        # keep track of when to trigger paragraph, chapter, and verse objects
        # para_track = 1 # keep counts of paragraphs
        prev_book = "Matt" # start at Genesis
        prev_chap = 1 # start at 1
        prev_verse = 1 # start at 1
        sentence_track = 1
        sentence_done = False
        clause_track = 1
        clause_done = False        
        wrdnum = 0 # start at 0
        this_chap = cv.node('chapter')
        this_sentence = cv.node('sentence')
        this_clause = cv.node('clause')        
        # this_para = cv.node('paragraph')
        this_verse = cv.node('verse')
        
        # iterate through words and construct objects
        for word in text:

            wrdnum += 1

            data = word.split('\t')
            # word_data, lemmas = data[:7], data[7:]

            word_data = data[:24] #the number here is the amount of columns
            morphology = ' '.join(data[24:]) #the number here is the amount of columns
            
            # segment out word data
            # bo_code, ref, brake, ketiv, qere, morph, strongs = word_data
            orig_order, book, chapter, verse, word, normalized, lemma, morph_form, morph_functional, strongs,lexeme_dict, gloss, sp, gn, nu, ps, case, tense, voice, mood, degree, extra, freq_lemma, dict_abc = word_data



            #try:
            #    verse = int(verse)
            #except ValueError:
            #    subverse = verse[-1:]
            #    verse = verse[:-1]

            if verse == "":
                print(f'{orig_order}: {verse} {subverse}')


            # strongs_lemma, anlex_lemma = ' '.join(lemmas).split('!') # reconstitute lemmas and split on !

            # chapt, verse, wrdnum = [int(v) for v in patts['section'].match(ref).groups()]

            # -- handle TF events --

            # detect book boundary
            if prev_book != book:

                # end subverse
                # cv.feature(this_subverse, subverse=prev_subverse)
                # cv.terminate(this_subverse)

                # end verse
                cv.feature(this_verse, verse=prev_verse)
                cv.terminate(this_verse)
                
                # end chapter
                cv.feature(this_chap, chapter=prev_chap)
                cv.terminate(this_chap)

                # end book
                cv.feature(this_book, book=prev_book)
                cv.terminate(this_book)
                
                # new book, chapter, verse, and subverse begin
                this_book = cv.node('book')
                prev_book = book
                this_chap = cv.node('chapter')
                prev_chap = chapter
                this_verse = cv.node('verse')
                prev_verse = verse
                #this_subverse = cv.node('subverse')
                #prev_subverse = subverse
                wrdnum = 1
            
            # detect chapter boundary
            elif prev_chap != chapter:

                # end subverse
                #cv.feature(this_subverse, subverse=prev_subverse)
                #cv.terminate(this_subverse)
                
                # end verse
                cv.feature(this_verse, verse=prev_verse)
                cv.terminate(this_verse)
                
                # end chapter
                cv.feature(this_chap, chapter=prev_chap)
                cv.terminate(this_chap)
                
                # new chapter, verse, and subverse begin
                this_chap = cv.node('chapter')
                prev_chap = chapter
                this_verse = cv.node('verse')
                prev_verse = verse
                #this_subverse = cv.node('subverse')
                #prev_subverse = subverse
                wrdnum = 1
            
            # detect verse boundary
            elif prev_verse != verse:

                # end subverse
                #cv.feature(this_subverse, subverse=prev_subverse)
                #cv.terminate(this_subverse)

                # end verse
                cv.feature(this_verse, verse=prev_verse)
                cv.terminate(this_verse)

                # new verse and subverse begin
                this_verse = cv.node('verse')
                prev_verse = verse
                #this_subverse = cv.node('subverse')
                #prev_subverse = subverse
                wrdnum = 1

            # detect subverse boundary
            #elif prev_subverse != subverse:
            #    cv.feature(this_subverse, subverse=prev_subverse)
            #    cv.terminate(this_subverse)
            #    this_subverse = cv.node('subverse')
            #    prev_subverse = subverse

                
            # detect paragraph boundary
            # if brake == 'P':
            #     cv.feature(this_para, para=para_track)
            #     cv.terminate(this_para)
            #     this_para = cv.node('paragraph') # start a new paragraph
            #     para_track += 1 # count paragraphs in the book
            
            if sentence_done:
                cv.feature(this_clause, clause=clause_track)
                cv.terminate(this_clause)
                cv.feature(this_sentence, sentence=sentence_track)
                cv.terminate(this_sentence)
                this_sentence = cv.node('sentence')
                sentence_track += 1
                this_clause = cv.node('clause')
                clause_track += 1

                sentence_done = False
                clause_done = False

            elif clause_done:
                cv.feature(this_clause, clause=clause_track)
                cv.terminate(this_clause)
                this_clause = cv.node('clause')
                clause_track += 1

                clause_done = False

            if text[-1:] == "." or text[-1:] == ";":
                sentence_done = True

            elif text[-1:] == "," or text[-1:] == "·":
                clause_done = True            
                
            # make word object
            this_word = cv.slot()
            cv.feature(this_word, 
                       orig_order=orig_order,
                       word=word,
                       normalized=normalized,
                       lemma=lemma,
                       morph_form=morph_form,
                       morph_functional=morph_functional,
                       strongs=strongs,
                       lexeme_dict=lexeme_dict,
                       gloss=gloss,
                       sp=sp,
                       gn=gn,
                       nu=nu,
                       ps=ps,
                       case=case,
                       tense=tense,
                       voice=voice,
                       mood=mood,
                       degree=degree,
                       extra=extra,
                       freq_lemma=freq_lemma,
                       dict_abc=dict_abc,

                       
                       # ketiv=ketiv, 
                       # qere=qere, 
                       # strongs=strongs, 
                    #    str_lem=strongs_lemma.strip(),
                    #    anlex_lem=anlex_lemma.strip()
                      )
            cv.terminate(this_word)
        
        # end book and its objects
        # - end subverse
        #cv.feature(this_subverse, subverse=prev_subverse)
        #cv.terminate(this_subverse)

        # - end verse
        cv.feature(this_verse, verse=prev_verse)
        cv.terminate(this_verse)
        
        # - end paragraph
        # cv.feature(this_para, para=para_track)
        # cv.terminate(this_para)
        
        # - end clause
        cv.feature(this_clause, clause=clause_track)
        cv.terminate(this_clause)

        # - end sentence
        cv.feature(this_sentence, sentence=sentence_track)
        cv.terminate(this_sentence)        
        
        # - end chapter
        cv.feature(this_chap, chapter=prev_chap)
        cv.terminate(this_chap)
        
        # - end book
        cv.feature(this_book, book=prev_book)
        cv.terminate(this_book)


slotType = 'word'
otext = {'fmt:text-orig-full':'{word} ',
         'sectionTypes':'book,chapter,verse',
         'sectionFeatures':'book,chapter,verse'}

generic = {'Name': 'NA1904',
           'Version': '1904',
           'Author': 'Nestle & Aland',
           'Editors': 'Ulrik Sandborg Petersen, Eliran Wong',
           'Converter': 'Oliver Glanz', 
           'Source:':'https://github.com/biblicalhumanities/Nestle1904/blob/master/morph/Nestle1904.csv',
           'Note':'?'}

intFeatures = {'chapter', 'verse'}

featureMeta = {'book': {'description': 'A book name'},
               'chapter': {'description': 'A chapter number'},
               'sentence': {'description': 'A sentence number'},
               'clause': {'description': 'A clause number'},               
               'verse': {'description': 'A verse number'},
               'word': {'description': 'Word with punctuation stripped'},
               'normalized': {'description': 'normalized word'},
               'lemma': {'description': 'ditionary entry form'},
               'morph_form': {'description': 'formal morphology'},
               'morph_functional': {'description': 'functional morphology'},
               'strongs': {'description': 'strongs number'},   
               'orig_order': {'description': 'original word order within the 1904 corpus'},
               'lexeme_dict': {'description': 'dictionary entry form'},
               'gloss': {'description': 'English gloss'},
               'sp': {'description': 'part of speech'},
               'gn': {'description': 'gender'},
               'nu': {'description': 'number'},
               'ps': {'description': 'person'},
               'case': {'description': 'case'},
               'tense': {'description': 'tense'},
               'voice': {'description': 'voice'},
               'mood': {'description': 'mood'},
               'degree': {'description': 'degree'},
               'extra': {'description': 'extra comments'},
               'freq_lemma': {'description': 'word frequency'},
               'dict_abc': {'description': 'alphabetic order of lexeme'},
               
               
               #'subverse': {'description': 'Subverse information'},
               # 'para': {'description': 'A paragraph number'},
               # 'ketiv': {'descrption': 'The text as it is written in the printed Tischendorf'},
               # 'qere': {'description': 'The text as the editor thinks it should have been'},
               # 'strongs': {'description': 'A word\'s number in Strongs'},
               # 'str_lem': {'description': 'Word lemma that corresponds to The NEW Strong\'sComplete Dictionary of Bible Words'},
               # 'anlex_lem': {'description': 'Word lemma that corresponds to Friberg, Friberg and Miller\'s ANLEX'}
              }


# configure metadata/output
version = '1904'
generic['Version'] = version

output = os.path.join(output_dirs, version)

print(f'Processing Version {version}')
output_dir = output_dirs.format(version=version)

TF = Fabric(locations=output_dir, silent=True)
cv = CV(TF)

cv.walk(director,
                slotType,
                otext=otext,
                generic=generic,
                intFeatures=intFeatures,
                featureMeta=featureMeta,
                warn=True,
                force=False,)

Processing Version 1904
  0.00s Importing data from walking through the source ...
   |     0.00s Preparing metadata... 
   |     0.00s No structure nodes will be set up
   |   SECTION   TYPES:    book, chapter, verse
   |   SECTION   FEATURES: book, chapter, verse
   |   STRUCTURE TYPES:    
   |   STRUCTURE FEATURES: 
   |   TEXT      FEATURES:
   |      |   text-orig-full       word
   |     0.00s OK
   |     0.00s Following director... 
	handling input\NTt2.txt...
   |     2.16s "edge" actions: 0
   |     2.16s "feature" actions: 146013
   |     2.16s "node" actions: 8234
   |     2.16s "resume" actions: 0
   |     2.16s "slot" actions: 137779
   |     2.17s "terminate" actions: 146013
   |         27 x "book" node 
   |        261 x "chapter" node 
   |          1 x "clause" node 
   |          1 x "sentence" node 
   |       7944 x "verse" node 
   |     137779 x "word" node  = slot type
   |     146013 nodes of all types
   |     2.19s OK
   |     0.00s Removing unlinked nodes .

True

# Testing New Features

In [1]:
# First, I have to laod different modules that I use for analyzing the data and for plotting:
import sys, os, collections
import pandas as pd
import numpy as np
import re
import csv
import seaborn as sns
import matplotlib.pyplot as plt; plt.rcdefaults()
from matplotlib.pyplot import figure
from collections import Counter

# Second, I have to load the Text Fabric app
from tf.fabric import Fabric
from tf.app import use

In [4]:
#LXX = use('CCATLXX/tf/1994_v1', hoist=globals())
NA2 = use('NA1904/tf/1904_v3', hoist=globals())

This is Text-Fabric 9.1.7
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

33 features found and 0 ignored


In [7]:
ExtraFeatures0 = '''
book book=Matt
  chapter chapter=1
    verse verse=1
      word
'''
ExtraFeatures0 = NA2.search(ExtraFeatures0)
NA2.show(ExtraFeatures0, start=1, end=1, condensed=True, colorMap={1:'pink'}, extraFeatures={'lemma','morph_form','morph_functional','normalized','strongs', 'case', 'degree', 'bol_dict_abc','extra', 'freq_lemma', 'gloss', 'gn', 'lemma', 'lexeme_dict', 'mood', 'nu', 'orig_order', 'ps', 'sp', 'strongs','tense','voice'})

  0.41s 8 results


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Converter-file" data-toc-modified-id="Converter-file-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Converter file</a></span><ul class="toc-item"><li><span><a href="#Oliver-Version-Working!" data-toc-modified-id="Oliver-Version-Working!-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Oliver Version Working!</a></span></li></ul></li><li><span><a href="#New-Feature-Development" data-toc-modified-id="New-Feature-Development-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>New Feature Development</a></span></li><li><span><a href="#Morphology" data-toc-modified-id="Morphology-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Morphology</a></span><ul class="toc-item"><li><span><a href="#MorphGNT:sblgnt-Data-Description" data-toc-modified-id="MorphGNT:sblgnt-Data-Description-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>MorphGNT:sblgnt Data Description</a></span><ul class="toc-item"><li><span><a href="#Adding-Part-of-Speech-sp" data-toc-modified-id="Adding-Part-of-Speech-sp-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Adding Part of Speech <code>sp</code></a></span></li></ul></li></ul></li><li><span><a href="#Word-Frequency" data-toc-modified-id="Word-Frequency-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Word Frequency</a></span></li><li><span><a href="#Converting-the-update-source-file" data-toc-modified-id="Converting-the-update-source-file-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Converting the update source file</a></span></li><li><span><a href="#Testing-New-Features" data-toc-modified-id="Testing-New-Features-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Testing New Features</a></span></li></ul></div>